# Análisis para tienda de alimentos
## Objetivo:
Estudiar el comportamiento de los usuarios para la aplicación de la empresa.

### Objetivos específicos:
* Análisis exploratorio:
  * Número de eventos y usuarios en los registros
  * Promedio de eventos por usuario
  * Periodo de los datos
  * Confirmación de 3 grupos experimentales

* Análisis de embudo:
  * Elaborar el embudo
  * Proporción de usuarios por cada evento
  * Proporción de usuarios que pasan a la siguiente etapa
  * Evaluar las etapas
  * Cantidad de usuarios que completan el viaje (primer evento hasta pago)

* Análisis de experimento A/A/B:
  * Número de usuarios por grupo
  * Evaluar diferencia entre grupos control (A/A)
  * Evaluar diferencia entre eventos para los grupos control
  * Comparar diferencia de grupos control con grupo con variante

## Diccionario de datos:
* ``EventName``: nombre del evento.
* ``DeviceIDHash``: identificador de usuario unívoco.
* ``EventTimestamp``: hora del evento.
* ``ExpId``: número de experimento. 246 y 247 son los grupos de control, y 248 es el grupo de prueba.

In [1]:
# importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# importar datasets
events = pd.read_csv('logs_exp_us.csv', sep='\t')

# Visualización general de los datos

In [3]:
# información general de 'logs_exp_us'
events.info()
print('\n', 'Hay', events.duplicated().sum(), 'valores duplicados')
print('\n', events.sample(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244126 entries, 0 to 244125
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   EventName       244126 non-null  object
 1   DeviceIDHash    244126 non-null  int64 
 2   EventTimestamp  244126 non-null  int64 
 3   ExpId           244126 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 7.5+ MB

 Hay 413 valores duplicados

                       EventName         DeviceIDHash  EventTimestamp  ExpId
166363         CartScreenAppear  7958840100052483837      1565020854    246
233336       OffersScreenAppear   933014534298828939      1565190698    248
145810       OffersScreenAppear  3024197421055968640      1564987968    247
185901         CartScreenAppear  4699227090087181127      1565080317    246
241297  PaymentScreenSuccessful  4369662623769092250      1565203845    248


### Comentario:
En el dataset no hay valores ausentes. Sin embargo, hay valores duplicados, por lo que se tiene que revisar estos valores, determinar su posible explicación y definir cómo deben de ser tratados. Por otro lado, hay que modificar el nombre de las columnas y en el caso de "EventTimestamp" los valores se encuentran en segundos por lo que hay que convertirlo a un tipo que se maneje con mayor facilidad.

Además puede resultar conveniente definir el tipo de datos para las columnas "DeviceIDHash" y "ExpId" ya que al ser variables categóricas pueden ser manejadas como `strings`.

# Limpieza de datos

In [4]:
# definir nombres de columnas
events.columns = ['event_name', 'uid', 'dt', 'exp_id']

In [8]:
# convertir a strings
events['uid'] = events['uid'].astype('str')
events['exp_id'] = events['exp_id'].astype('str')

# convertir a datetime
events['dt'] = pd.to_datetime(events['dt'], unit='s')

In [15]:
print(events[events.duplicated()])
print('\n', 'Número de usuarios:', events['uid'].nunique())

                     event_name                  uid                  dt  \
453            MainScreenAppear  5613408041324010552 2019-07-30 08:19:44   
2350           CartScreenAppear  1694940645335807244 2019-07-31 21:51:39   
3573           MainScreenAppear   434103746454591587 2019-08-01 02:59:37   
4076           MainScreenAppear  3761373764179762633 2019-08-01 03:47:46   
4803           MainScreenAppear  2835328739789306622 2019-08-01 04:44:01   
...                         ...                  ...                 ...   
242329         MainScreenAppear  8870358373313968633 2019-08-07 19:26:44   
242332  PaymentScreenSuccessful  4718002964983105693 2019-08-07 19:26:45   
242360  PaymentScreenSuccessful  2382591782303281935 2019-08-07 19:27:29   
242362         CartScreenAppear  2382591782303281935 2019-08-07 19:27:29   
242635         MainScreenAppear  4097782667445790512 2019-08-07 19:36:58   

       exp_id  
453       248  
2350      248  
3573      248  
4076      247  
4803   

### Comentario 
Después de haber estudiado los registros duplicados, me doy cuenta de que estos muestran que hay usuarios que realizaron varias veces la misma acción en el día, pero en horarios distintos, estos valores no deben de ser borrados ya que pueden servir para algunos tipos de análisis que involucren el comportamiento del usuario. Sin emabrgo, se debe de tomar en cuenta la presencia de estos a la hora de realizar los embudos y no aumentar registros por evento.